In [1]:
#@ Dataset for training gpt, we are using is tiny shakespares dataset:

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-02-12 16:11:06--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-02-12 16:11:07 (23.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
#@ Reading the file:

with open('input.txt', 'r', encoding='utf-8') as f:
  text=f.read()

In [3]:
print('length of dataset in characters:', len(text))

length of dataset in characters: 1115394


In [4]:
#@ inspecting:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
#@ Unique characters for the code:

chars=sorted(list(set(text)))
# print(chars)
vocab_size=len(chars)
# print(vocab_size) # unique characters len
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
#@ mapping from characters to integer:
str_int={ch:i for i, ch in enumerate(chars)}
int_str={i:ch for i, ch in enumerate(chars)}

encode=lambda s:[str_int[c] for c in s]
decode=lambda l: ''.join([int_str[i] for i in l])

print(encode('hello, i am prijal!'))
print(decode(encode('hello, i am prijal!')))

[46, 43, 50, 50, 53, 6, 1, 47, 1, 39, 51, 1, 54, 56, 47, 48, 39, 50, 2]
hello, i am prijal!


In [7]:
#@ Encoding the entire dataset:
import torch
data=torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
#@ Train-test split:
n=int(0.9*len(data)) # first 90% will be training
train_data=data[:n]
val_data=data[n:]

In [9]:
block_size=8 #seq len
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
  context=x[:t+1]
  target=y[t]
  print(f'when input is {context}, the target is: {target}')

when input is tensor([18]), the target is: 47
when input is tensor([18, 47]), the target is: 56
when input is tensor([18, 47, 56]), the target is: 57
when input is tensor([18, 47, 56, 57]), the target is: 58
when input is tensor([18, 47, 56, 57, 58]), the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [11]:
#@ constructing batch:

torch.manual_seed(147)
batch_size=4 #sequence for parallel processing
block_size=8 #max context len for prediction

def get_batch(split):
  data=train_data if split == 'train' else val_data
  ix=torch.randint(len(data)-block_size, (batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('*******')

for b in range(batch_size):
  for t in range(block_size):
    context=xb[b, :t+1]
    target=yb[b , t]
    print(f'input is {context.tolist()}, target:{target}')

inputs:
torch.Size([4, 8])
tensor([[43, 57,  6,  1, 40, 43, 41, 39],
        [ 1, 44, 50, 43, 41, 49, 43, 42],
        [ 1, 46, 43, 39, 56, 58,  1, 58],
        [24, 21, 26, 19, 14, 30, 27, 23]])
targets:
torch.Size([4, 8])
tensor([[57,  6,  1, 40, 43, 41, 39, 59],
        [44, 50, 43, 41, 49, 43, 42,  1],
        [46, 43, 39, 56, 58,  1, 58, 53],
        [21, 26, 19, 14, 30, 27, 23, 17]])
*******
input is [43], target:57
input is [43, 57], target:6
input is [43, 57, 6], target:1
input is [43, 57, 6, 1], target:40
input is [43, 57, 6, 1, 40], target:43
input is [43, 57, 6, 1, 40, 43], target:41
input is [43, 57, 6, 1, 40, 43, 41], target:39
input is [43, 57, 6, 1, 40, 43, 41, 39], target:59
input is [1], target:44
input is [1, 44], target:50
input is [1, 44, 50], target:43
input is [1, 44, 50, 43], target:41
input is [1, 44, 50, 43, 41], target:49
input is [1, 44, 50, 43, 41, 49], target:43
input is [1, 44, 50, 43, 41, 49, 43], target:42
input is [1, 44, 50, 43, 41, 49, 43, 42], target

In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(147)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits=self.token_embedding_table(idx) #(B,T,C)

    if targets is None:
      loss=None

    else:
      B, T, C =logits.shape
      logits=logits.view(B*T, C)
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx id (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      #getting the prediction:
      logits, loss=self(idx)

      #geting only the last time step:
      logits=logits[:, -1, :] #(B, C)

      #softmax:
      probs=F.softmax(logits, dim=-1)

      #sample from distribution:
      idx_next=torch.multinomial(probs, num_samples=1) #(B, 1)

      #append
      idx=torch.cat((idx, idx_next), dim=1) #(B, T+1)

    return idx

m=BigramLanguageModel(vocab_size)
logits, loss=m(xb, yb)

print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long),
                        max_new_tokens=100)[0].tolist()))



bnkvUpoZfSSPBq',grXmP3rQLAjJVwI-abnVdHhK''nW.yNKgzhSTPKCI:zpvWbNGfgNesjfNL,f.I:oMiknvy3:G GpD,tcY;

